In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd
import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

/home/boot/anaconda3/envs/XXX1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Node Classification

### Cora

In [15]:
dataset_sparse = Planetoid(root="/data/XXX/Pooling", name='Cora')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
class HierarchicalGCN_NOPool(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_NOPool, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x = self.down_convs[i](x, edge_index)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
        x = self.up_convs[-1](x, edge_index)
        return x
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()
        val_acc = eval_node_classifier(model, graph, val_mask)
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  
        else:
            patience_counter += 1  
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break
    return model, best_val_acc
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc
kf = KFold(n_splits=5, shuffle=True)
seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        model = HierarchicalGCN_NOPool(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        train_mask[train_index] = True
        test_mask[test_index] = True
        val_mask = test_mask  
        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)
        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')
    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  
    else:
        gpu_memory_usage = 0
    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')
for result in results:
    print(result)
mean_val_acc_values = [result['mean_val_acc'] for result in results]
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)
print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Seed 42, Fold 1 Val Acc: 0.887
Seed 42, Fold 2 Val Acc: 0.891
Seed 42, Fold 3 Val Acc: 0.882
Seed 42, Fold 4 Val Acc: 0.887
Early stopping at epoch 193 for fold 5
Seed 42, Fold 5 Val Acc: 0.906
Seed 42 Results: Mean Val Acc: 0.891, Time: 17.106 seconds, Memory: 31.477 MB, GPU Memory: 369.099 MB
Early stopping at epoch 188 for fold 1
Seed 123, Fold 1 Val Acc: 0.900
Seed 123, Fold 2 Val Acc: 0.904
Seed 123, Fold 3 Val Acc: 0.895
Seed 123, Fold 4 Val Acc: 0.891
Seed 123, Fold 5 Val Acc: 0.880
Seed 123 Results: Mean Val Acc: 0.894, Time: 16.858 seconds, Memory: 0.281 MB, GPU Memory: 369.099 MB
Seed 456, Fold 1 Val Acc: 0.897
Seed 456, Fold 2 Val Acc: 0.876
Seed 456, Fold 3 Val Acc: 0.902
Seed 456, Fold 4 Val Acc: 0.898
Early stopping at epoch 182 for fold 5
Seed 456, Fold 5 Val Acc: 0.911
Seed 456 Results: Mean Val Acc: 0.897, Time: 16.967 seconds, Memory: 0.286 MB, GPU Memory: 369.0

### CiteSeer

In [16]:
dataset_sparse = Planetoid(root="/data/XXX/Pooling", name='CiteSeer')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_NOPool(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_NOPool, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x = self.down_convs[i](x, edge_index)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
        x = self.up_convs[-1](x, edge_index)
        return x
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()
        val_acc = eval_node_classifier(model, graph, val_mask)
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  
        else:
            patience_counter += 1  
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break
    return model, best_val_acc
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc
kf = KFold(n_splits=5, shuffle=True)
seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        model = HierarchicalGCN_NOPool(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        train_mask[train_index] = True
        test_mask[test_index] = True
        val_mask = test_mask  
        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)
        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')
    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  
    else:
        gpu_memory_usage = 0
    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')
for result in results:
    print(result)
mean_val_acc_values = [result['mean_val_acc'] for result in results]
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)
print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
Early stopping at epoch 176 for fold 1
Seed 42, Fold 1 Val Acc: 0.794
Early stopping at epoch 179 for fold 2
Seed 42, Fold 2 Val Acc: 0.779
Early stopping at epoch 183 for fold 3
Seed 42, Fold 3 Val Acc: 0.777
Early stopping at epoch 181 for fold 4
Seed 42, Fold 4 Val Acc: 0.756
Early stopping at epoch 175 for fold 5
Seed 42, Fold 5 Val Acc: 0.759
Seed 42 Results: Mean Val Acc: 0.773, Time: 14.035 seconds, Memory: 0.312 MB, GPU Memory: 471.859 MB
Early stopping at epoch 184 for fold 1
Seed 123, Fold 1 Val Acc: 0.769
Early stopping at epoch 178 for fold 2
Seed 123, Fold 2 Val Acc: 0.787
Early stopping at epoch 179 for fold 3
Seed 123, Fold 3 Val Acc: 0.783
Early stopping at epoch 194 for fold 4
Seed 123, Fold 4 Val Acc: 0.756
Early stopping at epoch 180 for fold 5
Seed 123, Fold 5 Val Acc: 0.786
Seed 123 Results: Mean Val Acc: 0.776, Time: 14.145 seconds, Memory: 0.298 MB, GPU Memo

### Pubmed

In [17]:
dataset_sparse = Planetoid(root="/data/XXX/Pooling", name='PubMed')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_NOPool(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_NOPool, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x = self.down_convs[i](x, edge_index)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
        x = self.up_convs[-1](x, edge_index)
        return x
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()
        val_acc = eval_node_classifier(model, graph, val_mask)
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  
        else:
            patience_counter += 1  
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break
    return model, best_val_acc
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc
kf = KFold(n_splits=5, shuffle=True)
seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        model = HierarchicalGCN_NOPool(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        train_mask[train_index] = True
        test_mask[test_index] = True
        val_mask = test_mask  
        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)
        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')
    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  
    else:
        gpu_memory_usage = 0
    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')
for result in results:
    print(result)
mean_val_acc_values = [result['mean_val_acc'] for result in results]
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)
print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
Seed 42, Fold 1 Val Acc: 0.863
Seed 42, Fold 2 Val Acc: 0.866
Seed 42, Fold 3 Val Acc: 0.850
Seed 42, Fold 4 Val Acc: 0.860
Seed 42, Fold 5 Val Acc: 0.866
Seed 42 Results: Mean Val Acc: 0.861, Time: 16.454 seconds, Memory: 0.793 MB, GPU Memory: 532.677 MB
Seed 123, Fold 1 Val Acc: 0.853
Seed 123, Fold 2 Val Acc: 0.852
Seed 123, Fold 3 Val Acc: 0.870
Seed 123, Fold 4 Val Acc: 0.862
Seed 123, Fold 5 Val Acc: 0.866
Seed 123 Results: Mean Val Acc: 0.861, Time: 16.857 seconds, Memory: 0.782 MB, GPU Memory: 532.677 MB
Seed 456, Fold 1 Val Acc: 0.864
Seed 456, Fold 2 Val Acc: 0.859
Seed 456, Fold 3 Val Acc: 0.866
Seed 456, Fold 4 Val Acc: 0.858
Seed 456, Fold 5 Val Acc: 0.857
Seed 456 Results: Mean Val Acc: 0.861, Time: 15.540 seconds, Memory: 0.781 MB, GPU Memory: 532.677 MB
Seed 789, Fold 1 Val Acc: 0.860
Seed 789, Fold 2 Val Acc: 0.860
Seed 789, Fold 3 Val Acc: 0.864
Seed 789, Fo

### Cornell

In [10]:
from torch_geometric.datasets import WebKB
dataset_sparse = WebKB(root="/data/XXX/Pooling", name='Cornell')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_NOPool(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_NOPool, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x)  
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()
        val_acc = eval_node_classifier(model, graph, val_mask)
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  
        else:
            patience_counter += 1  
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break
    return model, best_val_acc
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc
kf = KFold(n_splits=5, shuffle=True)
seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        model = HierarchicalGCN_NOPool(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        train_mask[train_index] = True
        test_mask[test_index] = True
        val_mask = test_mask  
        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)
        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')
    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  
    else:
        gpu_memory_usage = 0
    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')
for result in results:
    print(result)
mean_val_acc_values = [result['mean_val_acc'] for result in results]
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)
print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[183, 1703], edge_index=[2, 298], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])
Early stopping at epoch 170 for fold 1
Seed 42, Fold 1 Val Acc: 0.541
Seed 42, Fold 2 Val Acc: 0.432
Early stopping at epoch 151 for fold 3
Seed 42, Fold 3 Val Acc: 0.459
Seed 42, Fold 4 Val Acc: 0.500
Early stopping at epoch 151 for fold 5
Seed 42, Fold 5 Val Acc: 0.444
Seed 42 Results: Mean Val Acc: 0.475, Time: 13.516 seconds, Memory: 0.253 MB, GPU Memory: 9965.666 MB
Early stopping at epoch 153 for fold 1
Seed 123, Fold 1 Val Acc: 0.486
Early stopping at epoch 179 for fold 2
Seed 123, Fold 2 Val Acc: 0.459
Early stopping at epoch 155 for fold 3
Seed 123, Fold 3 Val Acc: 0.459
Early stopping at epoch 152 for fold 4
Seed 123, Fold 4 Val Acc: 0.472
Early stopping at epoch 153 for fold 5
Seed 123, Fold 5 Val Acc: 0.500
Seed 123 Results: Mean Val Acc: 0.476, Time: 12.134 seconds, Memory: 0.219 MB, GPU Memory: 9965.666 MB
Seed 456, Fold 1 Val Acc: 0.459
Early stopping at epoch

### Texas

In [11]:
from torch_geometric.datasets import WebKB
dataset_sparse = WebKB(root="/data/XXX/Pooling", name='texas')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_NOPool(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_NOPool, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x)  
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()
        val_acc = eval_node_classifier(model, graph, val_mask)
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  
        else:
            patience_counter += 1  
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break
    return model, best_val_acc
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc
kf = KFold(n_splits=5, shuffle=True)
seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        model = HierarchicalGCN_NOPool(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        train_mask[train_index] = True
        test_mask[test_index] = True
        val_mask = test_mask  
        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)
        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')
    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  
    else:
        gpu_memory_usage = 0
    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')
for result in results:
    print(result)
mean_val_acc_values = [result['mean_val_acc'] for result in results]
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)
print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[183, 1703], edge_index=[2, 325], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])
Early stopping at epoch 153 for fold 1
Seed 42, Fold 1 Val Acc: 0.541
Early stopping at epoch 153 for fold 2
Seed 42, Fold 2 Val Acc: 0.595
Early stopping at epoch 151 for fold 3
Seed 42, Fold 3 Val Acc: 0.568
Early stopping at epoch 158 for fold 4
Seed 42, Fold 4 Val Acc: 0.583
Early stopping at epoch 153 for fold 5
Seed 42, Fold 5 Val Acc: 0.472
Seed 42 Results: Mean Val Acc: 0.552, Time: 12.252 seconds, Memory: 0.236 MB, GPU Memory: 9965.666 MB
Early stopping at epoch 165 for fold 1
Seed 123, Fold 1 Val Acc: 0.703
Early stopping at epoch 152 for fold 2
Seed 123, Fold 2 Val Acc: 0.459
Early stopping at epoch 155 for fold 3
Seed 123, Fold 3 Val Acc: 0.757
Early stopping at epoch 158 for fold 4
Seed 123, Fold 4 Val Acc: 0.611
Early stopping at epoch 154 for fold 5
Seed 123, Fold 5 Val Acc: 0.361
Seed 123 Results: Mean Val Acc: 0.578, Time: 12.182 seconds, Memory: 0.217 MB, G

### Wisconsin

In [12]:
from torch_geometric.datasets import WebKB
dataset_sparse = WebKB(root="/data/XXX/Pooling", name='wisconsin')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_NOPool(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_NOPool, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x)  
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()
        val_acc = eval_node_classifier(model, graph, val_mask)
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  
        else:
            patience_counter += 1  
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break
    return model, best_val_acc
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc
kf = KFold(n_splits=5, shuffle=True)
seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        model = HierarchicalGCN_NOPool(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        train_mask[train_index] = True
        test_mask[test_index] = True
        val_mask = test_mask  
        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)
        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')
    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  
    else:
        gpu_memory_usage = 0
    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')
for result in results:
    print(result)
mean_val_acc_values = [result['mean_val_acc'] for result in results]
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)
print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[251, 1703], edge_index=[2, 515], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10])
Early stopping at epoch 153 for fold 1
Seed 42, Fold 1 Val Acc: 0.549
Early stopping at epoch 161 for fold 2
Seed 42, Fold 2 Val Acc: 0.540
Early stopping at epoch 157 for fold 3
Seed 42, Fold 3 Val Acc: 0.580
Early stopping at epoch 156 for fold 4
Seed 42, Fold 4 Val Acc: 0.640
Early stopping at epoch 163 for fold 5
Seed 42, Fold 5 Val Acc: 0.500
Seed 42 Results: Mean Val Acc: 0.562, Time: 11.768 seconds, Memory: 0.236 MB, GPU Memory: 9965.666 MB
Early stopping at epoch 164 for fold 1
Seed 123, Fold 1 Val Acc: 0.510
Early stopping at epoch 151 for fold 2
Seed 123, Fold 2 Val Acc: 0.500
Seed 123, Fold 3 Val Acc: 0.580
Early stopping at epoch 153 for fold 4
Seed 123, Fold 4 Val Acc: 0.540
Early stopping at epoch 158 for fold 5
Seed 123, Fold 5 Val Acc: 0.640
Seed 123 Results: Mean Val Acc: 0.554, Time: 12.323 seconds, Memory: 0.226 MB, GPU Memory: 9965.666 MB
Early stopping a

### Github

In [21]:
from torch_geometric.datasets import GitHub
dataset_sparse = GitHub(root="/data/XXX/Pooling/GitHub")
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_NOPool(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_NOPool, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x = self.down_convs[i](x, edge_index)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
        x = self.up_convs[-1](x, edge_index)
        return x
def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  
    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()
        val_acc = eval_node_classifier(model, graph, val_mask)
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  
        else:
            patience_counter += 1  
        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break
    return model, best_val_acc
def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc
kf = KFold(n_splits=5, shuffle=True)
seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        model = HierarchicalGCN_NOPool(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        train_mask[train_index] = True
        test_mask[test_index] = True
        val_mask = test_mask  
        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)
        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')
    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  
    else:
        gpu_memory_usage = 0
    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')
for result in results:
    print(result)
mean_val_acc_values = [result['mean_val_acc'] for result in results]
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)
print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[37700, 128], edge_index=[2, 578006], y=[37700])
Seed 42, Fold 1 Val Acc: 0.859
Seed 42, Fold 2 Val Acc: 0.872
Seed 42, Fold 3 Val Acc: 0.861
Seed 42, Fold 4 Val Acc: 0.865
Seed 42, Fold 5 Val Acc: 0.866
Seed 42 Results: Mean Val Acc: 0.865, Time: 28.486 seconds, Memory: 1.391 MB, GPU Memory: 851.444 MB
Seed 123, Fold 1 Val Acc: 0.862
Seed 123, Fold 2 Val Acc: 0.862
Seed 123, Fold 3 Val Acc: 0.862
Seed 123, Fold 4 Val Acc: 0.871
Seed 123, Fold 5 Val Acc: 0.867
Seed 123 Results: Mean Val Acc: 0.865, Time: 28.681 seconds, Memory: 1.382 MB, GPU Memory: 851.444 MB
Seed 456, Fold 1 Val Acc: 0.857
Seed 456, Fold 2 Val Acc: 0.871
Seed 456, Fold 3 Val Acc: 0.865
Seed 456, Fold 4 Val Acc: 0.864
Seed 456, Fold 5 Val Acc: 0.866
Seed 456 Results: Mean Val Acc: 0.864, Time: 29.084 seconds, Memory: 1.375 MB, GPU Memory: 851.444 MB
Seed 789, Fold 1 Val Acc: 0.866
Seed 789, Fold 2 Val Acc: 0.866
Seed 789, Fold 3 Val Acc: 0.861
Seed 789, Fold 4 Val Acc: 0.864
Seed 789, Fold 5 Val Acc: 0.865
Seed

### ogbn-arxiv

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
from torch_geometric.loader import DataLoader
import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd
import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")
max_nodes = 150
dataset_sparse = PygNodePropPredDataset(root='/data/XXX/Pooling', name="ogbn-arxiv")
evaluator = Evaluator("ogbn-arxiv")
eval_metric = dataset_sparse.eval_metric

/home/boot/anaconda3/envs/XXX1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from torch_geometric.datasets import GitHub
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union
dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]
class HierarchicalGCN_NOPool(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, act=F.relu, sum_res=False):
        super(HierarchicalGCN_NOPool, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.sum_res = sum_res
        self.act = act
        channels = self.hidden_channels
        self.down_convs = torch.nn.ModuleList()
        self.down_convs.append(GCNConv(self.in_channels, channels))
        for i in range(self.depth):
            self.down_convs.append(GCNConv(channels, channels))
        in_channels = channels if sum_res else 2 * channels
        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(GCNConv(in_channels, channels))
        self.up_convs.append(GCNConv(channels, self.out_channels))
    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)
        x = self.down_convs[0](x, edge_index)
        x = F.relu(x) 
        xs = [x]
        edge_indices = [edge_index]
        for i in range(1, self.depth + 1):
            x = self.down_convs[i](x, edge_index)
            x = F.relu(x)
            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
        for i in range(self.depth):
            j = self.depth - 1 - i
            res = xs[j]
            edge_index = edge_indices[j]
            up = res
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = F.relu(x)
        x = self.up_convs[-1](x, edge_index)
        return x.log_softmax(dim=-1)
def train(model, data, train_idx, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()
    return loss.item()
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()
    out = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1, keepdim=True)
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']
    return train_acc, valid_acc, test_acc
seeds = [42, 123, 456]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []
for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
    val_accuracies = []
    test_accuracies = []
    start_time = time.time()
    tracemalloc.start()
    model = HierarchicalGCN_NOPool(
        in_channels=graph.num_features,
        hidden_channels=256,
        out_channels=dataset.num_classes,
        depth=2,
        act=F.relu,
        sum_res=False
    ).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    split_idx = dataset.get_idx_split()
    train_idx = split_idx['train'].to(device)
    for epoch in range(1, 501):
        loss = train(model, graph, split_idx['train'], optimizer)
        train_acc, valid_acc, test_acc = test(model, graph, split_idx, evaluator)
        val_accuracies.append(valid_acc)
        test_accuracies.append(test_acc)
        if epoch % 10 == 0:
            print(f'Seed: {seed:03d}, Epoch: {epoch:03d}, Loss: {loss:.4f}, '
                  f'Train Acc: {train_acc:.4f}, Val Acc: {valid_acc:.4f}, Test Acc: {test_acc:.4f}')
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    elapsed_time = end_time - start_time
    memory_usage = current / 10**6  
    peak_memory_usage = peak / 10**6  
    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.max_memory_allocated(device) / 1024**2  
        torch.cuda.reset_max_memory_allocated(device)
    else:
        gpu_memory_usage = 0
    val_accuracies_list.append(val_accuracies)
    times.append(elapsed_time)
    memories.append(memory_usage)
    gpu_memories.append(gpu_memory_usage)
    best_val_acc = max(val_accuracies)
    best_val_index = val_accuracies.index(best_val_acc)
    corresponding_test_acc = test_accuracies[best_val_index]
    results.append({
        'seed': seed,
        'best_val_acc': best_val_acc,
        'corresponding_test_acc': corresponding_test_acc,
        'elapsed_time': elapsed_time,
        'memory_usage': memory_usage,
        'gpu_memory_usage': gpu_memory_usage
    })
for result in results:
    print(f"Seed: {result['seed']}, Best Val Acc: {result['best_val_acc']:.4f}, "
          f"Corresponding Test Acc: {result['corresponding_test_acc']:.4f}, "
          f"Time: {result['elapsed_time']:.2f}s, Memory: {result['memory_usage']:.2f}MB, "
          f"GPU Memory: {result['gpu_memory_usage']:.2f}MB")
avg_val_acc = np.mean([result['best_val_acc'] for result in results])
avg_test_acc = np.mean([result['corresponding_test_acc'] for result in results])
avg_time = np.mean(times)
avg_memory = np.mean(memories)
avg_gpu_memory = np.mean(gpu_memories)
print(f"\nAverage Best Validation Accuracy: {avg_val_acc:.4f}")
print(f"Average Corresponding Test Accuracy: {avg_test_acc:.4f}")
print(f"Average Time: {avg_time:.2f}s")
print(f"Average Memory Usage: {avg_memory:.2f}MB")
print(f"Average GPU Memory Usage: {avg_gpu_memory:.2f}MB")

Seed: 042, Epoch: 010, Loss: 3.0037, Train Acc: 0.2701, Val Acc: 0.3018, Test Acc: 0.3320
Seed: 042, Epoch: 020, Loss: 2.3124, Train Acc: 0.4146, Val Acc: 0.4447, Test Acc: 0.4008
Seed: 042, Epoch: 030, Loss: 1.9781, Train Acc: 0.4658, Val Acc: 0.4647, Test Acc: 0.3967
Seed: 042, Epoch: 040, Loss: 1.8068, Train Acc: 0.5143, Val Acc: 0.4991, Test Acc: 0.4390
Seed: 042, Epoch: 050, Loss: 1.6787, Train Acc: 0.5385, Val Acc: 0.5242, Test Acc: 0.4645
Seed: 042, Epoch: 060, Loss: 1.6096, Train Acc: 0.5526, Val Acc: 0.5350, Test Acc: 0.4680
Seed: 042, Epoch: 070, Loss: 1.5674, Train Acc: 0.5642, Val Acc: 0.5406, Test Acc: 0.4735
Seed: 042, Epoch: 080, Loss: 1.5374, Train Acc: 0.5702, Val Acc: 0.5468, Test Acc: 0.4775
Seed: 042, Epoch: 090, Loss: 1.5089, Train Acc: 0.5770, Val Acc: 0.5541, Test Acc: 0.4887
Seed: 042, Epoch: 100, Loss: 1.5275, Train Acc: 0.5771, Val Acc: 0.5617, Test Acc: 0.5005
Seed: 042, Epoch: 110, Loss: 1.4752, Train Acc: 0.5829, Val Acc: 0.5584, Test Acc: 0.4924
Seed: 042,